In [33]:
X = []
y = []
with open('src/amazon_reviews.txt') as inputfile:
    for line in inputfile:
        X.append(line.strip().split('   ')[0][11:])
        y.append(int(line.strip().split('   ')[0][9]))

Commençons par préparer les données afin qu'elles soient utilisables par un algorithme.

In [41]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

def token(text):
    words = set(text_to_word_sequence(text))
    return one_hot(text, 10000)
    
X_token = [token(X[i]) for i in range(len(X))]

In [66]:
for i in range(len(y)):
    if y[i] == 2:
        y[i]=0

In [67]:
np.unique(np.array(y))

array([0, 1])

Quel genre de commentaires avons-nous finalement?

In [53]:
X[0]

'Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I\'m in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life\'s hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"'

In [44]:
print(len(X[0]))
len(X_token[0])

533


105

In [39]:
print(len(X))
print(len(y))
print(len(X_token))

10000
10000
10000


In [60]:
lenght = [len(X_token[i]) for i in range(len(X_token))]

print(sum(lenght)/len(lenght))
print(max(lenght))
print(min(lenght))

80.9356
207
12


In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_token, y, test_size=0.2, random_state=42)

In [70]:
from keras.preprocessing import sequence
X_train = sequence.pad_sequences(X_train,
                                    value=0,
                                    padding='post', 
                                    maxlen=100) 

X_test = sequence.pad_sequences(X_test,
                                   value=0,
                                   padding='post', 
                                   maxlen=100) 

Appliquons lui maintenant un réseau de neuronnes de type RNN

In [71]:
from keras.models import Sequential
from keras.layers import GRU, LSTM, Dense, Embedding


def GRU_RNN():

    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=32, input_length=100))

    model.add(GRU(units=32, return_sequences=True))
    model.add(GRU(units=32, return_sequences=False))
    model.add(Dense(units=1, activation='sigmoid'))

    
    return model

In [72]:
model = GRU_RNN()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 24s 3ms/step - loss: 0.6912 - acc: 0.5300 - val_loss: 0.6847 - val_acc: 0.5625
Epoch 2/10
8000/8000 [==============================] - 22s 3ms/step - loss: 0.6379 - acc: 0.6114 - val_loss: 0.6595 - val_acc: 0.6160
Epoch 3/10
8000/8000 [==============================] - 21s 3ms/step - loss: 0.4989 - acc: 0.7399 - val_loss: 0.4972 - val_acc: 0.7685
Epoch 4/10
8000/8000 [==============================] - 22s 3ms/step - loss: 0.2931 - acc: 0.8821 - val_loss: 0.4399 - val_acc: 0.8115
Epoch 5/10
8000/8000 [==============================] - 22s 3ms/step - loss: 0.1857 - acc: 0.9409 - val_loss: 0.5339 - val_acc: 0.8115
Epoch 6/10
8000/8000 [==============================] - 22s 3ms/step - loss: 0.1237 - acc: 0.9639 - val_loss: 0.5496 - val_acc: 0.8125
Epoch 7/10
8000/8000 [==============================] - 22s 3ms/step - loss: 0.0869 - acc: 0.9790 - val_loss: 0.6307 - val_acc: 0.7990
Epoch 8

Nous constatons donc un gros overfitting... C'est le risque avec ce genre de modèle. Néanmoins, nous pourrions dans un premier temps modifier nos paramètres afin d'améliorer le réultat et combattre l'overfitting.